<a href="https://colab.research.google.com/github/NicoleSimas/Reescrita_ic/blob/main/IC_Reescrita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Projeto: Projeções de Bioenergia até 2050 em quatro cenários com Simulações de Monte Carlo
Base de Dados: Artigo "Global bioenergy potentials projections for 2050"
Autores do artigo: M. R. Errera, T. A. C. Dias, D. M. Y. Maya, E. E. S. Lora

- Última revisão: 25/02/2025
- 4 cenários apresentados: CS (atual), BUS (2050 sem mudanças no estilo de vida atual),
OPT (2050 com tendências otimistas), FAR (2050 com adaptação total)
- As variáveis principais são geradas a partir de distribuições normais com base nos parâmetros do artigo
- Os resultados são apresentados em histogramas

Ferramentas utilizadas:
- Python 3.12.1
- Pandas, NumPy, Plotly
"""

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Carregar os valores padrão (usado para vr2 e vr3)
df_VP = pd.read_csv('valorespadrao.csv', sep=';')
df_VP = df_VP.apply(pd.to_numeric, errors='coerce')

# Função para cálculo do fornecimento total de bionergia em 2050/biofonte/cenário [EJ]
def fproj14mc5(params=None):

    # Definição de dimensões das matrizes
    N_BS = 10  # Número de biofontes
    N_S = 4   # Número de cenários

    # Inicialização das variáveis de ajuste de projeções
    vr1 = np.zeros((N_BS, N_S))
    vr2 = np.zeros((N_BS, N_S))
    vr3 = np.zeros((N_BS, N_S))
    BS2050ks = np.zeros((N_BS, N_S))  # bioenergia [EJ] 2050
    BS_CS = np.zeros((N_BS, 1))  # EJ

    # Parâmetros fixos
    BS_CSf = df_VP.loc[0:10, 'BS_CSf'].values # Participação de cada biofonte no CS [%]
    TFLA = df_VP.loc[0, 'TFLA_CS':'TFLA_FAR'].values  # Projeção de áreas cultiváveis [Mha]
    OfCal_L = df_VP.loc[0, 'OfCal_L_CS':'OfCal_L_FAR'].values  # Oferta calórica ("Livestock", animais) [Gcal]
    OfCal_C = df_VP.loc[0, 'OfCal_C_CS':'OfCal_C_FAR'].values  # Oferta calórica ("Crops", vegetais) [Gcal]
    ECLA = df_VP.loc[0, 'ECLA_CS':'ECLA_FAR'].values  # Área de culturas energéticas [Mha]

    if params is None:
        # Valores padrão quando não há parâmetros fornecidos
        BSG_CS = df_VP.loc[0, 'BSG_CS']  # Total de bioenergia em 2018/ base para CS [EJ]
        pEFLA = df_VP.loc[0, 'pEFLA']  # Fração florestal permitida para exploração de culturas lenhosas [%] - arbitrado para o cenário atual
        Pop = df_VP.loc[0, 'Pop_CS':'Pop_FAR'].values  # População mundial/cenário [bilhões]
        vr2[:, :] = df_VP.loc[:, 'vr2_CS':'vr2_FAR'].values
        vr3[:, :] = df_VP.loc[:, 'vr3_CS':'vr3_FAR'].values
    else:
        # Parâmetros utilizados para os cálculos, "unpacking"
        (
            BSG_CS, pEFLA, Pop_CS, Pop_BUS, Pop_OPT, Pop_FAR,
            vr2_1a6_CS, vr2_1a6_BUS, vr2_1a6_OPT, vr2_1a6_FAR,
            vr2_7a10, vr3_3_OPT, vr3_3_FAR, vr3_4_OPT, vr3_4_FAR,
            vr3_5_OPT, vr3_5_FAR, vr3_8a9_OPT, vr3_8a9_FAR
        ) = params

        # População por cenário
        Pop = np.array([Pop_CS, Pop_BUS, Pop_OPT, Pop_FAR])

        # Aplicação dos valores aos vetores de variação
        vr2[0:6, :] = np.array([vr2_1a6_CS, vr2_1a6_BUS, vr2_1a6_OPT, vr2_1a6_FAR])
        vr2[6:10, 1:4] = vr2_7a10
        vr3[2, 2:4] = np.array([vr3_3_OPT, vr3_3_FAR])
        vr3[3, 2:4] = np.array([vr3_4_OPT, vr3_4_FAR])
        vr3[4, 2:4] = np.array([vr3_5_OPT, vr3_5_FAR])
        vr3[7:9, 2:4] = np.array([vr3_8a9_OPT, vr3_8a9_FAR])
        vr3[9, 2] = vr3[3, 2]
        vr3[9, 3] = vr3[3, 3]

    # Cálculo dos valores de correção de vr1
    vr1[:, 0] = 1.0  # Unitário para CS

    # Correção por biofonte
    EFLA = pEFLA * TFLA
    vr1[0:3, :] = EFLA / EFLA[0]  # Derivados de madeira, florestas energéticas/indústria/área explorável
    vr1[3, :] = OfCal_L / OfCal_L[0]  # Agricultura animal
    vr1[4, :] = OfCal_C / OfCal_C[0]  # Agricultura vegetal
    vr1[5, :] = ECLA / ECLA[0]  # Culturas energéticas
    vr1[6:10, :] = Pop / Pop[0]  # Resíduos energéticos

    # Fornecimento total de bionergia em 2050/biofonte/cenário [EJ]
    BS_CS = BSG_CS * BS_CSf.reshape(-1, 1) / 100
    BS2050ks = BS_CS * vr1 * (1 + vr2) * (1 + vr3)
    BE2050s = BS2050ks.sum(axis=0)

    return BS2050ks, BE2050s

# Gerar amostras baseadas na distribuição normal/ Modelo de Monte Carlo
def generate_samples(mu, error_margin, n):
    faixa = error_margin * mu
    sd = faixa / 7  # Margem de erro de ±3 desvios padrão
    return np.random.randn(n) * sd + mu

# Carregar os valores de mu(média) e error_margem(margem de erro) para a simulação
df_S = pd.read_csv('mc_simul.csv', sep=';')
df_S = df_S.apply(pd.to_numeric, errors='coerce')

# Inicializar variáveis
n_simul = 10001
N_BS = 10
N_S = 4
data_simulated = {}
gravar = False

# Criar amostras simuladas
for col in df_S.columns:
    mu = df_S.loc[0, col]
    error_margin = df_S.loc[1, col]
    data_simulated[col] = generate_samples(mu, error_margin, n_simul)

# Adicionar coluna fixa de população no CS
data_simulated["Pop_CS"] = np.full(n_simul, 7.7)

# Criar DataFrame com as amostras simuladas
df_simul = pd.DataFrame(data_simulated)

# Inicializar arrays para Monte Carlo
BE2050smc = np.zeros((n_simul, N_S))
BS2050ksmc = np.zeros((N_BS, N_S, n_simul))

# Preparar parâmetros para simulação
BSG_CS = df_simul['BSG_CS'].values
pEFLA = df_simul['pEFLA'].values
vr2_1a6 = df_simul.loc[:, 'vr2_1a6_CS':'vr2_1a6_FAR'].values
vr2_7a10 = df_simul.loc[:, 'vr2_7a10_BUS_OPT_FAR'].values
Pop = df_simul.loc[:, 'Pop_CS':'Pop_FAR'].values
vr3_3 = df_simul.loc[:, 'vr3_3_OPT':'vr3_3_FAR'].values
vr3_4 = df_simul.loc[:, 'vr3_4_OPT':'vr3_4_FAR'].values
vr3_5 = df_simul.loc[:, 'vr3_5_OPT':'vr3_5_FAR'].values
vr3_8a9 = df_simul.loc[:, 'vr3_8a9_OPT':'vr3_8a9_FAR'].values

# Simulações de Monte Carlo
for i in range(n_simul):
    finput = [
        BSG_CS[i], pEFLA[i], *Pop[i, :], *vr2_1a6[i, :], vr2_7a10[i], *vr3_3[i, :],
        *vr3_4[i, :], *vr3_5[i, :], *vr3_8a9[i, :]
    ]
# '*' é o operador de unpacking, passa os valores pra a função como argumentos individuais, ao invés de listas
    BS2050ks, BE2050s = fproj14mc5(finput)
    BS2050ksmc[:, :, i] = BS2050ks
    BE2050smc[i, :] = BE2050s

# Estatísticas sobre as projeções de bioenergia/2050
BE2050smed = np.mean(BE2050smc, axis=0)     # Média/cenário
BE2050sstd = np.std(BE2050smc, axis=0)      # Desvio padrão/cenário
BE2050median = np.median(BE2050smc, axis=0) # Mediana/cenário
BS2050mean = np.mean(BS2050ksmc, axis=2)    # Média/biofonte/cenáriocenário

# Ajustes de exibição
np.set_printoptions(suppress=True, precision=6)

# Função para plotar histogramas com Plotly
def plot_histograms():
    # Criar subplots
    fig = make_subplots(
        rows=7, cols=3,
        subplot_titles=(
            'BSG_CS', 'pEFLA', '',
            'vr2 (1 to 6) - CS', 'vr2 (1 to 6) - BUS @2050', '',
            'vr2 (1 to 6) - OPT @2050', 'vr2 (1 to 6) - FAR @2050', '',
            'vr2 (7 to 10) - BUS, OPT, FAR @2050', '', '',
            'Population 2050 - BUS', 'Population 2050 - OPT', 'Population 2050 - FAR',
            'CS', 'BUS @2050', '', 'OPT @2050', 'FAR @2050', ''
        ),
        vertical_spacing=0.1, horizontal_spacing=0.1
    )

    # Adicionar histogramas
    fig.add_trace(go.Histogram(x=BSG_CS, nbinsx=21, marker_color='blue', marker_line_color='white'), row=1, col=1)
    fig.add_trace(go.Histogram(x=pEFLA, nbinsx=21, marker_color='cyan', marker_line_color='white'), row=1, col=2)

    fig.add_trace(go.Histogram(x=vr2_1a6[:, 0], nbinsx=21, marker_color='blue', marker_line_color='white'), row=2, col=1)
    fig.add_trace(go.Histogram(x=vr2_1a6[:, 1], nbinsx=21, marker_color='red', marker_line_color='white'), row=2, col=2)

    fig.add_trace(go.Histogram(x=vr2_1a6[:, 2], nbinsx=21, marker_color='green', marker_line_color='white'), row=3, col=1)
    fig.add_trace(go.Histogram(x=vr2_1a6[:, 3], nbinsx=21, marker_color='magenta', marker_line_color='white'), row=3, col=2)

    fig.add_trace(go.Histogram(x=vr2_7a10, nbinsx=21, marker_color='cyan', marker_line_color='white'), row=4, col=1)

    fig.add_trace(go.Histogram(x=Pop[:, 1], nbinsx=21, marker_color='red', marker_line_color='white'), row=5, col=1)
    fig.add_trace(go.Histogram(x=Pop[:, 2], nbinsx=21, marker_color='green', marker_line_color='white'), row=5, col=2)
    fig.add_trace(go.Histogram(x=Pop[:, 3], nbinsx=21, marker_color='magenta', marker_line_color='white'), row=5, col=3)

    fig.add_trace(go.Histogram(x=BE2050smc[:, 0], nbinsx=21, marker_color='#0072BD', marker_line_color='white'), row=6, col=1)
    fig.add_trace(go.Histogram(x=BE2050smc[:, 1], nbinsx=21, marker_color='#A2142F', marker_line_color='white'), row=6, col=2)

    fig.add_trace(go.Histogram(x=BE2050smc[:, 2], nbinsx=21, marker_color='#008080', marker_line_color='white'), row=7, col=1)
    fig.add_trace(go.Histogram(x=BE2050smc[:, 3], nbinsx=21, marker_color='#7E2F8E', marker_line_color='white'), row=7, col=2)

    # Atualizar layout para proporções iguais
    fig.update_layout(
        height=2000, width=1500,
        showlegend=False,
        bargap=0.1,
        barmode='overlay',
        template="plotly_white"
    )

    # Ajustar transparência para visualização de sobreposições
    for trace in fig.data:
        trace.opacity = 0.8

    #Adicionar definição dos gráficos
    fig.update_xaxes(title_text="Bionergy [EJ]", row=1, col=1)
    fig.update_xaxes(title_text="Exploitation Share [%]", row=1, col=2)
    fig.update_xaxes(title_text="Productivity Gain [%]", row=2, col=1)
    fig.update_xaxes(title_text="Productivity Gain [%]", row=2, col=2)
    fig.update_xaxes(title_text="Productivity Gain [%]", row=3, col=1)
    fig.update_xaxes(title_text="Productivity Gain [%]", row=3, col=2)
    fig.update_xaxes(title_text="Productivity Gain [%]", row=4, col=1)
    fig.update_xaxes(title_text="Billions of inhabitants", row=5, col=1)
    fig.update_xaxes(title_text="Billions of inhabitants", row=5, col=2)
    fig.update_xaxes(title_text="Billions of inhabitants", row=5, col=3)
    fig.update_xaxes(title_text="Primary Bionergy [EJ]", row=6, col=1)
    fig.update_xaxes(title_text="Primary Bionergy [EJ]", row=6, col=2)
    fig.update_xaxes(title_text="Primary Bionergy [EJ]", row=7, col=1)
    fig.update_xaxes(title_text="Primary Bionergy [EJ]", row=7, col=2)

    fig.show()

# Plotar histogramas
plot_histograms()

# Gravar soluções:
if gravar:
    np.savetxt('proj14mc5.txt', BE2050smc)
    np.savetxt('proj14Pop.txt', Pop)
    np.savetxt('proj14vr21a6.txt', vr2_1a6)
    np.savetxt('proj14vr27a10.txt', vr2_7a10)
    np.savetxt('proj14vr3_3.txt', vr3_3)
    np.savetxt('proj14vr3_4.txt', vr3_4)
    np.savetxt('proj14vr3_5.txt', vr3_5)
    np.savetxt('proj14vr3_8a9.txt', vr3_8a9)
    np.savetxt('proj14pEFLA.txt', pEFLA)
    np.savetxt('proj14BSG_CS.txt', BSG_CS)
    np.savetxt('proj14BS2050mean.txt', BS2050mean)
